### Dataset creation for World Development Indicators 

In [1]:
import pandas as pd
import numpy as np
import wbgapi as wb # api for fetching world bank data

pd.set_option('display.max_rows', None) # show all rows

In [2]:
# get info about wbgapi
wb.source.info()

id,name,code,concepts,lastupdated
1,Doing Business,DBS,3,2021-08-18
2,World Development Indicators,WDI,3,2024-03-28
3,Worldwide Governance Indicators,WGI,3,2023-09-29
5,Subnational Malnutrition Database,SNM,3,2016-03-21
6,International Debt Statistics,IDS,4,2024-02-29
11,Africa Development Indicators,ADI,3,2013-02-22
12,Education Statistics,EDS,3,2023-10-12
13,Enterprise Surveys,ESY,3,2022-03-25
14,Gender Statistics,GDS,3,2024-03-04
15,Global Economic Monitor,GEM,3,2024-01-17


In [3]:
# create DF for World Development Indicators
indicators = wb.series.info(db = 2)
df_ind = pd.DataFrame(indicators.table(),columns=['Indicator Code', 'Indicator Name'])

In [4]:
df_ind.head()

,Indicator Code,Indicator Name
0,AG.AGR.TRAC.NO,"Agricultural machinery, tractors"
1,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer produc...
2,AG.CON.FERT.ZS,Fertilizer consumption (kilograms per hectare ...
3,AG.LND.AGRI.K2,Agricultural land (sq. km)
4,AG.LND.AGRI.ZS,Agricultural land (% of land area)


#### Get columns/variables of interest
Note: this part has to be done manually by checking the column 'Indicator Name' of df_ind.

In [5]:
# search for the variables, e.g. 'Life expectancy':
df_ind[df_ind['Indicator Name'].str.contains('Life expectancy')]

,Indicator Code,Indicator Name
1298,SP.DYN.LE00.FE.IN,"Life expectancy at birth, female (years)"
1299,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)"
1300,SP.DYN.LE00.MA.IN,"Life expectancy at birth, male (years)"


In [6]:
# dictionary for the variables of interest

dict_map_indi = {'SP.DYN.LE00.IN': 'Life expectancy at birth, total (years)', 'SP.DYN.TFRT.IN': 'Fertility rate, total (births per woman)',
                 'SP.DYN.CBRT.IN': 'Birth rate, crude (per 1,000 people)', 'SP.DYN.CDRT.IN': 'Death rate, crude (per 1,000 people)', 
                 'SP.DYN.IMRT.IN': 'Mortality rate, infant (per 1,000 live births)', 'SP.POP.TOTL': 'Population, total',
                 'SP.POP.0014.TO': 'Population ages 0-14, total', 'SP.POP.1564.TO': 'Population ages 15-64, total', 
                 'SP.POP.65UP.TO': 'Population ages 65 and above, total', 'NY.GDP.PCAP.CD': 'GDP per capita (current US$)',
                 'SM.POP.NETM': 'Net migration'}

In [7]:
# get country name and county code
df_country = pd.DataFrame(wb.economy.info().table() , columns=['Country Code', 'Country Name', 'region','incomeLevel'])
df_country.head()

,Country Code,Country Name,region,incomeLevel
0,ABW,Aruba,LCN,HIC
1,AFE,Africa Eastern and Southern,,
2,AFG,Afghanistan,SAS,LIC
3,AFW,Africa Western and Central,,
4,AGO,Angola,SSF,LMC


#### Create data set by fetching data with the indicators of interest from the api
Note: fetching the data and building the data set takes a few minutes to run

In [8]:
def alldata(indicators):
    '''
    Builds the data set by fetching data with the indicators of interest from the api
    ###### Note: running this function takes a few minutes to run !!! #######
    '''
    
    dfs = []
    
    for icode, iname in dict_map_indi.items():
        df = wb.data.DataFrame(series=icode)
        df.insert(0,'Indicator Name', iname)
        df.insert(1,'Indicator Code', icode)
        dfs.append(df)
        
    result_df = pd.concat(dfs)

    # merge df_country and result_df
    merged_df = df_country.drop('incomeLevel',axis=1).merge(result_df, left_on='Country Code', right_index=True)
    
    # rename column years into integers
    merged_df.rename(columns=lambda x: int(x[2:]) if x.startswith('YR') else x, inplace=True)

    return merged_df    

In [9]:
df = alldata(dict_map_indi)

In [10]:
df.head()

,Country Code,Country Name,region,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,ABW,Aruba,LCN,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,64.152,64.537,64.752,65.132,65.294,...,75.601,75.683,75.617,75.903,76.072,76.248,75.723,74.626,NaN,NaN
0,ABW,Aruba,LCN,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,4.820,4.655,4.471,4.271,4.059,...,2.148,1.972,1.953,1.839,1.587,1.486,1.325,1.180,NaN,NaN
0,ABW,Aruba,LCN,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,33.883,32.831,31.649,30.416,29.147,...,13.533,12.428,12.300,11.530,9.881,9.138,8.102,7.193,NaN,NaN
0,ABW,Aruba,LCN,"Death rate, crude (per 1,000 people)",SP.DYN.CDRT.IN,6.594,6.456,6.394,6.275,6.250,...,7.729,7.968,8.336,8.414,8.597,8.778,9.690,11.235,NaN,NaN
0,ABW,Aruba,LCN,"Mortality rate, infant (per 1,000 live births)",SP.DYN.IMRT.IN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Unpivot data set to create DF in long format

In [11]:
def wide_to_long(df):
  '''
  creates df in long format from the wdi_data data set in wide format
  '''
  # loop through columns 
  for col in df['Indicator Code'].unique():
    # create helper df only with Indicator Code for the respective iteration
    df_help = df[df["Indicator Code"] == col]
      
    # set variable value_name (used below in pd.melt function)
    value_name = df_help.iloc[0]['Indicator Name']

    # drop columns 'Indicator Name', 'Indicator Code
    df_help = df_help.drop(['Indicator Name', 'Indicator Code'], axis=1)
      
    if col == df['Indicator Code'].unique()[0]: # first iteration will create new df df_melt
      df_melt = pd.melt(df_help, id_vars=['Country Name', 'Country Code', 'region'], var_name='Year', value_name=value_name)
    else: # next ierations will add new column to df_melt
      df_melt[value_name] =  pd.melt(df_help, id_vars=['Country Name', 'Country Code', 'region'], var_name='Year', value_name=value_name)[value_name]
    df_melt['Year'] = df_melt.Year.astype(int)

  return df_melt

In [12]:
df_final = wide_to_long(df)
df_final.head()

,Country Name,Country Code,region,Year,"Life expectancy at birth, total (years)","Fertility rate, total (births per woman)","Birth rate, crude (per 1,000 people)","Death rate, crude (per 1,000 people)","Mortality rate, infant (per 1,000 live births)","Population, total","Population ages 0-14, total","Population ages 15-64, total","Population ages 65 and above, total",GDP per capita (current US$),Net migration
0,Aruba,ABW,LCN,1960,64.152000,4.820000,33.883000,6.594000,NaN,54608.0,23553.0,29759.0,1296.0,NaN,0.0
1,Africa Eastern and Southern,AFE,,1960,44.085552,6.724125,47.438554,20.918108,NaN,130692579.0,57766140.0,69041513.0,3884927.0,141.385955,-90849.0
2,Afghanistan,AFG,SAS,1960,32.535000,7.282000,50.340000,31.921000,NaN,8622466.0,3589290.0,4788899.0,244277.0,62.369375,2606.0
3,Africa Western and Central,AFW,,1960,37.845152,6.458448,47.325476,26.363067,NaN,97256290.0,40188607.0,53856591.0,3211093.0,107.053706,-847.0
4,Angola,AGO,SSF,1960,38.211000,6.708000,51.026000,26.659000,NaN,5357195.0,2262702.0,2929489.0,165004.0,NaN,-43749.0
